In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.stats as stats
import numpy as np

In [15]:
REPLICATION = True

file_path_add = ''
if REPLICATION:
    file_path_add = 'replication/'

basemodelname = 'gpt2-large'

In [16]:
def get_correlation(prompts,metric_name, user_study = 'globalAMT'):

    user_ratings = []
    perplexity = []
    text = []
    does = []
    for row in prompts:
        if f'user_score_{user_study}' not in row:
            continue
        user_score = row[f'user_score_{user_study}']

        text.append(row['aa'] + ' ' + row['aci'])
        user_ratings.append(user_score)
        perplexity.append(row[metric_name])
        if user_score >= 0:
            does.append(True)
        else:
            does.append(False)
    
    does_ids = [i for i in range(len(does)) if does[i] == True]
    donts_ids = [i for i in range(len(does)) if does[i] == False]
    user_ratings_does = np.array(user_ratings)[does_ids]
    perplexity_does = np.array(perplexity)[does_ids]

    user_ratings_donts = np.array(user_ratings)[donts_ids]
    perplexity_donts = np.array(perplexity)[donts_ids]
    print(stats.pearsonr(user_ratings, perplexity))


## Perplexity difference

In [11]:
def get_finetuned_gpt2_results(test_type, train_data):
    if train_data == 'wvs':
        train_data = '_on_wvs'
    else:
        train_data = '_on_pew'
    prompts = pickle.load(open(f'../data/{file_path_add}MoRT_actions/prompts_{basemodelname}_{test_type}{train_data}_pair_logprob_userstudy_globalAMT_style_mv_at_end_use_last_token_True.p',
      'rb'))
    get_correlation(prompts, 'log prob difference')
    

In [17]:
for train_data in ['wvs','pew']:
    for test_type in ['random','country_based','topic based']:
        print(train_data, test_type)
        get_finetuned_gpt2_results(test_type, train_data)
        

wvs random
PearsonRResult(statistic=0.6668343613861194, pvalue=3.6225427837075014e-14)
wvs country_based
PearsonRResult(statistic=0.6997530684466258, pvalue=5.51079052648661e-16)
wvs topic based
PearsonRResult(statistic=0.738690479025508, pvalue=1.756747386594492e-18)
pew random
PearsonRResult(statistic=0.8230124455672642, pvalue=8.23480424851103e-26)
pew country_based
PearsonRResult(statistic=0.8362134780129264, pvalue=2.587727658222531e-27)
pew topic based
PearsonRResult(statistic=0.5086458689437663, pvalue=6.550060451376035e-08)
